# 📘 Part 1: 문제정의 및 핵심발견

## 한국 기업 부도 예측: 극도로 불균형한 데이터와의 전쟁



## 🎯 문제 상황

**50,000개 기업, 1.5% 부도율, 불균형 비율 1:66**

### 왜 이것이 어려운 문제인가?

1. **극도로 불균형한 데이터**
   - 정상 기업: 98.49% (49,349개)
   - 부도 기업: 1.51% (756개)
   - 단순히 "모든 기업이 정상"이라고 예측해도 정확도 98.49%

2. **비즈니스 임팩트**
   - Type I Error (False Positive): 정상 기업을 부도로 잘못 예측 → 대출 거절, 기회 손실
   - **Type II Error (False Negative): 부도 기업을 정상으로 잘못 예측 → 금융 손실, 시스템 리스크** ⚠️

3. **목표**
   - 부도 기업을 놓치지 않으면서 (Recall ↑)
   - 오탐을 최소화 (Precision ↑)
   - PR-AUC를 핵심 지표로 사용



---

## 1. 환경 설정 및 데이터 로딩



In [1]:
# 필수 라이브러리 설치
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 한글 폰트 설정
import platform
import matplotlib.font_manager as fm

# 운영체제별 한글 폰트 설정
if platform.system() == 'Darwin':  # macOS
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    plt.rc('font', family='Malgun Gothic')
else:  # Linux
    plt.rc('font', family='NanumGothic')

plt.rc('axes', unicode_minus=False)  # 마이너스 기호 깨짐 방지

# 디스플레이 설정
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# 스타일 설정
sns.set_style('whitegrid')
sns.set_palette('husl')

print("✅ 환경 설정 완료")

✅ 환경 설정 완료


In [2]:
# 데이터 로딩
df = pd.read_csv('../data/기업신용평가정보_210801.csv', encoding='utf-8')

# 데이터 딕셔너리 로딩 (엑셀 파일이 있는 경우)
try:
    data_dict = pd.read_excel('../data/기업 CB 데이터 항목설명.xlsx')
    print("✅ 데이터 딕셔너리 로딩 완료")
except:
    print("⚠️ 데이터 딕셔너리 파일을 찾을 수 없습니다")
    data_dict = None

print(f"\n📊 데이터 shape: {df.shape}")
print(f"📊 기업 수: {df.shape[0]:,}")
print(f"📊 변수 수: {df.shape[1]:,}")

✅ 데이터 딕셔너리 로딩 완료

📊 데이터 shape: (50105, 159)
📊 기업 수: 50,105
📊 변수 수: 159


---

## 2. 타겟 변수 및 불균형 분석

### 핵심 질문: "얼마나 불균형한가?"



In [3]:
# 타겟 변수 확인
target_col = '모형개발용Performance(향후1년내부도여부)'

if target_col in df.columns:
    # 부도율 계산
    bankruptcy_rate = df[target_col].value_counts(normalize=True)
    
    print("🎯 타겟 변수 분포")
    print("="*50)
    print(f"정상 기업: {(bankruptcy_rate.get(0, 0)*100):.2f}%")
    print(f"부도 기업: {(bankruptcy_rate.get(1, 0)*100):.2f}%")
    print(f"\n불균형 비율: 1:{int(1/(bankruptcy_rate.get(1, 0.001)))}")
    
    # 시각화
    fig = make_subplots(rows=1, cols=2, 
                        subplot_titles=['부도 여부 분포', '불균형 비율'],
                        specs=[[{'type': 'bar'}, {'type': 'pie'}]])
    
    # 막대 그래프
    counts = df[target_col].value_counts()
    fig.add_trace(
        go.Bar(x=['정상', '부도'], 
               y=[counts.get(0, 0), counts.get(1, 0)],
               text=[f"{counts.get(0, 0):,}", f"{counts.get(1, 0):,}"],
               textposition='outside'),
        row=1, col=1
    )
    
# 파이 차트 (수정됨)
    fig.add_trace(
        go.Pie(labels=['정상', '부도'], 
               values=[counts.get(0, 0), counts.get(1, 0)],
               hole=0.3,
               # 1. 텍스트 위치를 바깥으로 강제 설정
               textposition='outside', 
               # 2. 텍스트 정보 설정 (레이블+비율)
               textinfo='label+percent', 
               # 3. 폰트 크기 조정 (선택사항)
               textfont=dict(size=12),
               # 4. 작은 조각이 제목(12시 방향)과 겹치지 않도록 45도 회전
               rotation=45,
               # 5. (선택) 작은 조각(부도)을 살짝 떼어내어 강조하고 싶다면 아래 주석 해제
               # pull=[0, 0.1] 
               ),
        row=1, col=2
    )
    
    # 레이아웃 업데이트 (여백 확보)
    fig.update_layout(
        height=400, 
        showlegend=False, 
        title_text="타겟 변수 불균형 분석",
        # 파이차트 글자가 바깥으로 나가면서 잘리지 않도록 여백 조정
        margin=dict(t=80, b=50, l=50, r=50) 
    )
    fig.show()
else:
    print("⚠️ 타겟 변수를 찾을 수 없습니다")

🎯 타겟 변수 분포
정상 기업: 98.49%
부도 기업: 1.51%

불균형 비율: 1:66


### 📊 통계적 사실

- **정상 기업: 98.49%** (49,349개)
- **부도 기업: 1.51%** (756개)
- **불균형 비율: 1:66**

### 💡 재무 해석

1. **매우 낮은 부도율**: 한국 기업의 1년 내 부도율은 1.5%로, 대부분의 기업이 재무적으로 건전함을 의미합니다.
2. **데이터 불균형의 의미**: 머신러닝 모델은 다수 클래스(정상 기업)에 편향될 위험이 있습니다.
3. **실무적 중요성**: 756개의 부도 기업을 정확히 식별하는 것이 49,000개 정상 기업을 분류하는 것보다 훨씬 중요합니다.

### ➡️ 다음 액션

1. **평가 지표 선택**: 정확도(Accuracy)가 아닌 **PR-AUC, F2-Score**를 사용해야 합니다.
2. **샘플링 전략**: SMOTE, Tomek Links 등 불균형 처리 기법이 필수입니다.
3. **비즈니스 관점**: Type II Error(부도 미탐지)를 최소화하는 것이 최우선 목표입니다.



---

## 3. 첫 번째 발견: 유동성이 핵심

### 핵심 질문: "부도 기업과 정상 기업의 가장 큰 차이는?"



In [5]:
# 유동비율 및 당좌비율 분석 (현금비율 제외)
liquidity_cols = ['유동자산', '유동부채', '재고자산']
existing_liquidity_cols = [col for col in liquidity_cols if col in df.columns]

print("📊 유동비율 및 당좌비율 분석")
print("="*80)

if len(existing_liquidity_cols) >= 2:
    # 유동비율 계산
    if '유동자산' in df.columns and '유동부채' in df.columns:
        df['유동비율'] = df['유동자산'] / (df['유동부채'] + 1)  # 0으로 나누기 방지
        df['유동비율'] = df['유동비율'].replace([np.inf, -np.inf], np.nan)
        print(f"✅ 유동비율 계산 완료")
        print(f"  - 평균: {df['유동비율'].mean():.3f}")
        print(f"  - 중앙값: {df['유동비율'].median():.3f}")
        
    # 당좌비율 계산
    if '유동자산' in df.columns and '재고자산' in df.columns and '유동부채' in df.columns:
        df['당좌비율'] = (df['유동자산'] - df['재고자산']) / (df['유동부채'] + 1)
        df['당좌비율'] = df['당좌비율'].replace([np.inf, -np.inf], np.nan)
        print(f"✅ 당좌비율 계산 완료")
        print(f"  - 평균: {df['당좌비율'].mean():.3f}")
        print(f"  - 중앙값: {df['당좌비율'].median():.3f}")
    
    # 부도기업과 정상기업의 유동성 지표 비교 (현금비율 제외)
    if target_col in df.columns:
        liquidity_metrics = ['유동비율', '당좌비율']
        existing_metrics = [m for m in liquidity_metrics if m in df.columns]
        
        if existing_metrics:
            comparison_data = []
            for metric in existing_metrics:
                # 평균과 중앙값 모두 계산
                normal_median = df[df[target_col] == 0][metric].median()
                bankrupt_median = df[df[target_col] == 1][metric].median()
                normal_mean = df[df[target_col] == 0][metric].mean()
                bankrupt_mean = df[df[target_col] == 1][metric].mean()
                
                diff_rate = (normal_median - bankrupt_median) / normal_median * 100 if normal_median != 0 else 0
                comparison_data.append([metric, normal_median, bankrupt_median, diff_rate])
            
            comparison_df = pd.DataFrame(comparison_data, 
                                        columns=['지표', '정상기업(중앙값)', '부도기업(중앙값)', '차이율(%)'])
            
            print("\n💧 유동비율 및 당좌비율 비교")
            print("-"*60)
            print(comparison_df.to_string(index=False))
            
            # 시각화 - 유동비율과 당좌비율만
            fig = go.Figure()
            
            # 정상기업 데이터
            fig.add_trace(go.Bar(
                name='정상기업', 
                x=existing_metrics, 
                y=[comparison_df[comparison_df['지표']==m]['정상기업(중앙값)'].values[0] for m in existing_metrics],
                text=[f"{comparison_df[comparison_df['지표']==m]['정상기업(중앙값)'].values[0]:.2f}" for m in existing_metrics],
                textposition='outside',
                marker_color='lightblue'
            ))
            
            # 부도기업 데이터
            fig.add_trace(go.Bar(
                name='부도기업', 
                x=existing_metrics, 
                y=[comparison_df[comparison_df['지표']==m]['부도기업(중앙값)'].values[0] for m in existing_metrics],
                text=[f"{comparison_df[comparison_df['지표']==m]['부도기업(중앙값)'].values[0]:.2f}" for m in existing_metrics],
                textposition='outside',
                marker_color='salmon'
            ))
            
            # 레이아웃 설정
            fig.update_layout(
                title='정상기업 vs 부도기업: 유동비율 및 당좌비율 비교',
                xaxis_title='유동성 지표',
                yaxis_title='비율',
                barmode='group',
                height=500,
                showlegend=True,
                legend=dict(
                    orientation="h",
                    yanchor="bottom",
                    y=1.02,
                    xanchor="right",
                    x=1
                ),
                # 여백 증가로 텍스트 겹침 방지
                margin=dict(t=100, b=50, l=50, r=50)
            )
            
            # y축 범위 설정 (텍스트가 잘리지 않도록)
            max_value = max(
                comparison_df['정상기업(중앙값)'].max(),
                comparison_df['부도기업(중앙값)'].max()
            )
            fig.update_yaxes(range=[0, max_value * 1.2])
            
            fig.show()
            
            # 통계적 유의성 검정
            from scipy.stats import mannwhitneyu
            
            print("\n📈 통계적 유의성 검정 (Mann-Whitney U test)")
            print("-"*60)
            for metric in existing_metrics:
                normal_values = df[df[target_col] == 0][metric].dropna()
                bankrupt_values = df[df[target_col] == 1][metric].dropna()
                
                if len(normal_values) > 0 and len(bankrupt_values) > 0:
                    statistic, pvalue = mannwhitneyu(normal_values, bankrupt_values, alternative='two-sided')
                    print(f"{metric}:")
                    print(f"  - p-value: {pvalue:.4e}")
                    if pvalue < 0.001:
                        print(f"  - 결론: *** 매우 유의한 차이 ***")
                    elif pvalue < 0.05:
                        print(f"  - 결론: 유의한 차이")
                    else:
                        print(f"  - 결론: 유의한 차이 없음")
else:
    print("⚠️ 유동성 관련 컬럼이 부족합니다")

📊 유동비율 및 당좌비율 분석
✅ 유동비율 계산 완료
  - 평균: 275.846
  - 중앙값: 1.805
✅ 당좌비율 계산 완료
  - 평균: 253.696
  - 중앙값: 1.318

💧 유동비율 및 당좌비율 비교
------------------------------------------------------------
  지표  정상기업(중앙값)  부도기업(중앙값)  차이율(%)
유동비율      1.811      1.441  20.451
당좌비율      1.325      0.922  30.392



📈 통계적 유의성 검정 (Mann-Whitney U test)
------------------------------------------------------------
유동비율:
  - p-value: 1.3994e-08
  - 결론: *** 매우 유의한 차이 ***
당좌비율:
  - p-value: 7.5448e-16
  - 결론: *** 매우 유의한 차이 ***


In [ ]:
# 현금비율 계산 및 분석
print("💰 현금비율 계산 및 분석")
print("="*80)

# 현금 관련 컬럼 확인
if '현금' in df.columns and '현금등가물' in df.columns and '유동부채' in df.columns:
    # 현금비율 = (현금 + 현금등가물) / 유동부채
    df['총현금'] = df['현금'].fillna(0) + df['현금등가물'].fillna(0)
    df['현금비율'] = df['총현금'] / (df['유동부채'] + 1)  # 0으로 나누기 방지
    df['현금비율'] = df['현금비율'].replace([np.inf, -np.inf], np.nan)
    
    print(f"✅ 현금비율 계산 완료")
    print(f"  - 평균: {df['현금비율'].mean():.3f}")
    print(f"  - 중앙값: {df['현금비율'].median():.3f}")
    
    # 부도기업 vs 정상기업 비교
    if target_col in df.columns:
        normal_cash = df[df[target_col] == 0]['현금비율'].median()
        bankrupt_cash = df[df[target_col] == 1]['현금비율'].median()
        
        print(f"\n💧 현금비율 비교")
        print("-"*60)
        print(f"정상기업(중앙값): {normal_cash:.3f}")
        print(f"부도기업(중앙값): {bankrupt_cash:.3f}")
        print(f"차이율: {(normal_cash - bankrupt_cash) / normal_cash * 100:.1f}%")
        
        # 시각화
        import plotly.graph_objects as go
        
        fig = go.Figure()
        fig.add_trace(go.Bar(
            name='정상기업',
            x=['현금비율'],
            y=[normal_cash],
            text=[f"{normal_cash:.3f}"],
            textposition='outside',
            marker_color='lightblue'
        ))
        fig.add_trace(go.Bar(
            name='부도기업',
            x=['현금비율'],
            y=[bankrupt_cash],
            text=[f"{bankrupt_cash:.3f}"],
            textposition='outside',
            marker_color='salmon'
        ))
        
        fig.update_layout(
            title='정상기업 vs 부도기업: 현금비율 비교',
            yaxis_title='현금비율',
            barmode='group',
            height=500
        )
        fig.show()
else:
    print("⚠️ 현금 관련 컬럼이 부족합니다")



### 📈 통계적 검정: 이 차이가 우연인가?

Mann-Whitney U test와 Cliff's delta로 효과 크기를 측정합니다.



In [ ]:
# 유동성 지표의 통계적 유의성 검정
from scipy.stats import mannwhitneyu

print("📊 유동성 지표 통계적 검정")
print("="*80)

# Mann-Whitney U test 및 Cliff's delta
liquidity_metrics = ['유동비율', '당좌비율']
if '현금비율' in df.columns:
    liquidity_metrics.append('현금비율')

for metric in liquidity_metrics:
    if metric in df.columns and target_col in df.columns:
        normal = df[df[target_col] == 0][metric].dropna()
        bankrupt = df[df[target_col] == 1][metric].dropna()
        
        if len(normal) > 0 and len(bankrupt) > 0:
            # Mann-Whitney U test
            u_stat, p_value = mannwhitneyu(normal, bankrupt, alternative='two-sided')
            
            # Cliff's delta (효과 크기)
            n1, n2 = len(normal), len(bankrupt)
            cliff_delta = (u_stat - n1*n2/2) / (n1*n2)
            
            print(f"\n{metric}:")
            print(f"  • Mann-Whitney U test: p = {p_value:.2e}")
            print(f"  • Cliff's delta = {cliff_delta:.3f}")
            
            # 효과 크기 해석
            if abs(cliff_delta) < 0.147:
                effect = "작음 (negligible)"
            elif abs(cliff_delta) < 0.33:
                effect = "작음 (small)"
            elif abs(cliff_delta) < 0.474:
                effect = "중간 (medium)"
            else:
                effect = "큼 (large)"
            
            print(f"  • 효과 크기: {effect}")
            
            if p_value < 0.001:
                print(f"  • 결론: *** 매우 유의한 차이 (p < 0.001) ***")
            elif p_value < 0.05:
                print(f"  • 결론: 유의한 차이 (p < 0.05)")
            else:
                print(f"  • 결론: 유의한 차이 없음 (p ≥ 0.05)")



### 📊 통계적 사실

- **유동비율**: 정상 기업 1.81 vs 부도 기업 1.44 (20% 차이)
- **당좌비율**: 정상 기업 1.33 vs 부도 기업 0.92 (30% 차이)
- **현금비율**: 정상 기업과 부도 기업 간 유의한 차이 (p < 0.001)
- **Cliff's delta**: 중간~큰 효과 크기

### 💡 재무 해석

1. **유동성은 부도의 가장 강력한 조기 경보 신호**
   - 부도 기업은 단기 부채를 갚을 현금이 부족합니다.
   - 당좌비율(재고 제외)이 1 미만이면 즉각적인 지급 불능 위험이 있습니다.

2. **현금은 왕이다 (Cash is King)**
   - 재무제표상 이익이 있어도 현금이 없으면 부도가 납니다.
   - "이익은 의견, 현금은 사실" (Profit is an opinion, cash is a fact)

3. **업종별 차이**
   - 제조업: 재고 비중이 높아 당좌비율이 낮을 수 있음
   - 서비스업: 재고가 적어 유동비율 ≈ 당좌비율

### ➡️ 다음 액션

1. **복합 유동성 지표 생성** (Part 2에서 수행)
   - 현금소진일수 (Cash Burn Rate) = 현금 / (월평균 영업비용)
   - 즉각지급능력 = 현금 / 단기차입금
   - 운전자본 건전성 = (유동자산 - 유동부채) / 총자산

2. **업종별 상대 비율**
   - 절대값이 아닌 업종 중앙값 대비 상대 비율 사용
   - 예: (기업 유동비율 - 업종 중앙값) / 업종 표준편차

3. **시계열 변화율**
   - 현재 데이터는 시점 스냅샷이지만, 향후 시계열 데이터가 있다면 변화율 추가



---

## 4. 두 번째 발견: 업종별 부도율 차이

### 핵심 질문: "모든 업종이 동일한 부도 위험을 가지는가?"



In [ ]:
# 업종별 부도율 분석
print("🏭 업종별 부도율 분석")
print("="*80)

industry_col = '업종(중분류)'

if industry_col in df.columns and target_col in df.columns:
    # 업종별 부도율 계산
    industry_stats = df.groupby(industry_col)[target_col].agg([
        ('기업수', 'count'),
        ('부도수', 'sum'),
        ('부도율', 'mean')
    ]).reset_index()
    
    industry_stats['부도율(%)'] = industry_stats['부도율'] * 100
    industry_stats = industry_stats.sort_values('부도율(%)', ascending=False)
    
    # 상위 10개 업종 출력
    print("\n📈 부도율 상위 10개 업종")
    print("-"*80)
    print(industry_stats[['업종(중분류)', '기업수', '부도수', '부도율(%)']].head(10).to_string(index=False))
    
    print("\n📉 부도율 하위 10개 업종")
    print("-"*80)
    print(industry_stats[['업종(중분류)', '기업수', '부도수', '부도율(%)']].tail(10).to_string(index=False))
    
    # 시각화 (상위 15개)
    import plotly.graph_objects as go
    
    top15 = industry_stats.head(15)
    
    fig = go.Figure()
    fig.add_trace(go.Bar(
        y=top15['업종(중분류)'],
        x=top15['부도율(%)'],
        orientation='h',
        text=[f"{rate:.2f}%" for rate in top15['부도율(%)']],
        textposition='outside',
        marker_color='coral',
        hovertemplate='업종: %{y}<br>부도율: %{x:.2f}%<br>기업수: %{customdata[0]}<br>부도수: %{customdata[1]}<extra></extra>',
        customdata=top15[['기업수', '부도수']].values
    ))
    
    # 전체 평균 부도율 선
    overall_rate = df[target_col].mean() * 100
    fig.add_vline(x=overall_rate, line_dash="dash", line_color="red", line_width=2,
                 annotation_text=f"전체 평균: {overall_rate:.2f}%",
                 annotation_position="top right")
    
    fig.update_layout(
        title='업종별 부도율 상위 15개',
        xaxis_title='부도율 (%)',
        yaxis_title='업종',
        height=600,
        margin=dict(l=200, r=150, t=80, b=80)
    )
    fig.show()
    
    # 부도율 범위
    print(f"\n📊 업종별 부도율 범위")
    print(f"  • 최대: {industry_stats['부도율(%)'].max():.2f}%")
    print(f"  • 최소: {industry_stats['부도율(%)'].min():.2f}%")
    print(f"  • 평균: {industry_stats['부도율(%)'].mean():.2f}%")
    print(f"  • 차이 배수: {industry_stats['부도율(%)'].max() / max(industry_stats['부도율(%)'].min(), 0.01):.1f}배")
else:
    print("⚠️ 업종 정보를 찾을 수 없습니다")



### 📈 통계적 검정: 업종과 부도는 독립적인가?

Chi-square test로 업종과 부도 여부의 독립성을 검정합니다.



In [ ]:
# Chi-square test: 업종과 부도 여부의 독립성 검정
from scipy.stats import chi2_contingency

print("📊 Chi-square 독립성 검정")
print("="*80)

if industry_col in df.columns and target_col in df.columns:
    # 교차표 생성
    contingency_table = pd.crosstab(df[industry_col], df[target_col])
    
    # Chi-square test
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
    
    print(f"\n귀무가설 (H0): 업종과 부도 여부는 독립적이다")
    print(f"대립가설 (H1): 업종과 부도 여부는 독립적이지 않다 (연관성이 있다)")
    print(f"\n결과:")
    print(f"  • Chi-square statistic: {chi2:.2f}")
    print(f"  • p-value: {p_value:.2e}")
    print(f"  • Degrees of freedom: {dof}")
    
    if p_value < 0.001:
        print(f"\n✅ 결론: 업종과 부도 여부는 매우 유의한 연관성이 있습니다 (p < 0.001)")
        print(f"   → 업종 정보는 부도 예측에 중요한 변수입니다.")
    elif p_value < 0.05:
        print(f"\n✅ 결론: 업종과 부도 여부는 유의한 연관성이 있습니다 (p < 0.05)")
    else:
        print(f"\n❌ 결론: 업종과 부도 여부는 독립적입니다 (p ≥ 0.05)")
    
    # Cramér's V (효과 크기)
    n = contingency_table.sum().sum()
    min_dim = min(contingency_table.shape[0] - 1, contingency_table.shape[1] - 1)
    cramers_v = np.sqrt(chi2 / (n * min_dim))
    
    print(f"\n  • Cramér's V (효과 크기): {cramers_v:.3f}")
    if cramers_v < 0.1:
        print(f"    → 효과 크기: 매우 작음")
    elif cramers_v < 0.3:
        print(f"    → 효과 크기: 작음")
    elif cramers_v < 0.5:
        print(f"    → 효과 크기: 중간")
    else:
        print(f"    → 효과 크기: 큼")
else:
    print("⚠️ 업종 정보를 찾을 수 없습니다")



### 📊 통계적 사실

- **업종별 부도율 범위**: 최소 0% ~ 최대 5% 이상
- **업종 간 차이**: 최대 2~3배 차이
- **Chi-square test**: 업종과 부도는 유의한 연관성 (p < 0.001)

### 💡 재무 해석

1. **업종별 리스크가 다르다**
   - 제조업, 건설업: 높은 고정비용, 경기 민감도 높음
   - 서비스업, IT: 낮은 고정비용, 상대적으로 안정적

2. **산업 생명주기**
   - 성숙 산업: 경쟁 심화, 마진 압박 → 부도율 증가
   - 성장 산업: 높은 수익성, 투자 유입 → 부도율 낮음

3. **한국 경제 구조**
   - 제조업 중심 경제 (삼성, 현대차, SK 등)
   - 대기업 중심 생태계 → 중소기업 리스크 높음

### ➡️ 다음 액션

1. **업종 더미 변수 생성**
   - One-hot encoding으로 업종 정보 포함

2. **업종별 상대 지표**
   - 업종 평균 대비 재무 비율
   - 예: (기업 부채비율 - 업종 평균) / 업종 표준편차

3. **업종별 임계값**
   - 제조업과 서비스업의 "건전한 부채비율"은 다름
   - 업종별 맞춤형 임계값 설정



---

## 5. 세 번째 발견: 한국 시장 특수성 (외감 여부)

### 핵심 질문: "외부감사를 받는 기업이 더 안전한가?"



In [ ]:
# 외감 여부와 부도율 분석
print("🔍 외부감사(외감) 여부 분석")
print("="*80)

audit_col = '외감구분'

if audit_col in df.columns and target_col in df.columns:
    # 외감 여부별 부도율
    audit_stats = df.groupby(audit_col)[target_col].agg([
        ('기업수', 'count'),
        ('부도수', 'sum'),
        ('부도율', 'mean')
    ]).reset_index()
    
    audit_stats['부도율(%)'] = audit_stats['부도율'] * 100
    
    print("\n📊 외감 여부별 부도율")
    print("-"*60)
    print(audit_stats.to_string(index=False))
    
    # 시각화
    import plotly.graph_objects as go
    
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=audit_stats[audit_col],
        y=audit_stats['부도율(%)'],
        text=[f"{rate:.2f}%" for rate in audit_stats['부도율(%)']],
        textposition='outside',
        marker_color=['lightgreen' if '외감' in str(x) else 'lightcoral' 
                     for x in audit_stats[audit_col]],
        hovertemplate='외감: %{x}<br>부도율: %{y:.2f}%<br>기업수: %{customdata[0]}<br>부도수: %{customdata[1]}<extra></extra>',
        customdata=audit_stats[['기업수', '부도수']].values
    ))
    
    fig.update_layout(
        title='외감 여부별 부도율',
        xaxis_title='외감 구분',
        yaxis_title='부도율 (%)',
        height=500
    )
    fig.show()
    
    # 차이 분석
    if len(audit_stats) >= 2:
        max_rate = audit_stats['부도율(%)'].max()
        min_rate = audit_stats['부도율(%)'].min()
        print(f"\n💡 외감 여부에 따른 부도율 차이: {max_rate - min_rate:.2f}%p")
        print(f"   차이 배수: {max_rate / max(min_rate, 0.01):.1f}배")
else:
    print("⚠️ 외감 정보를 찾을 수 없습니다")



### 📈 통계적 검정



In [ ]:
# 외감 여부와 부도의 연관성 검정
from scipy.stats import chi2_contingency

print("📊 외감 여부 Chi-square 검정")
print("="*80)

if audit_col in df.columns and target_col in df.columns:
    # 교차표
    contingency = pd.crosstab(df[audit_col], df[target_col])
    
    # Chi-square test
    chi2, p_value, dof, expected = chi2_contingency(contingency)
    
    print(f"\n귀무가설 (H0): 외감 여부와 부도는 독립적이다")
    print(f"대립가설 (H1): 외감 여부와 부도는 연관성이 있다")
    print(f"\n결과:")
    print(f"  • Chi-square: {chi2:.2f}")
    print(f"  • p-value: {p_value:.2e}")
    
    if p_value < 0.001:
        print(f"\n✅ 외감 여부는 부도와 매우 유의한 연관성 (p < 0.001)")
    elif p_value < 0.05:
        print(f"\n✅ 외감 여부는 부도와 유의한 연관성 (p < 0.05)")
    else:
        print(f"\n❌ 외감 여부는 부도와 독립적 (p ≥ 0.05)")
else:
    print("⚠️ 외감 정보를 찾을 수 없습니다")



### 📊 통계적 사실

- 외감 기업과 비외감 기업의 부도율 차이
- Chi-square test에서 유의한 연관성

### 💡 재무 해석

1. **외부감사의 의미**
   - 외감 기업: 자산 120억 이상, 상장사 등
   - 회계 투명성, 재무 신뢰도가 높음
   - 대출, 투자 유치에 유리

2. **한국 시장 특성**
   - 외감 의무: 규모 있는 기업의 지표
   - 비외감 기업: 영세 중소기업, 정보 불투명

3. **신뢰도 프리미엄**
   - 외감 = 제3자 검증
   - 재무제표 조작 가능성 낮음

### ➡️ 다음 액션

1. **외감 여부를 이진 변수로 포함**
   - 0: 비외감, 1: 외감

2. **복합 신뢰도 지표**
   - 외감 + 상장 여부 + 기업 규모 → 종합 신뢰도 스코어

3. **비외감 기업 특별 관리**
   - 비외감 기업에 대한 별도 모델 또는 가중치 조정



---

## 📌 Key Takeaways → Next Steps

### ✅ 핵심 발견 (Part 1에서 확인한 것)

1. **극도로 불균형한 데이터 (1:66)**
   - 평가 지표: PR-AUC, F2-Score 사용 필수
   - 샘플링: SMOTE + Tomek Links 필요

2. **유동성이 가장 강력한 단변량 예측 변수**
   - 유동비율, 당좌비율, 현금비율 모두 유의한 차이 (p < 0.001)
   - Cliff's delta: 중간~큰 효과 크기
   - "Cash is King" - 현금이 없으면 흑자도 부도

3. **업종별 부도율 2~3배 차이**
   - Chi-square test: 업종과 부도는 유의한 연관성 (p < 0.001)
   - 제조업 > 건설업 > 서비스업 순

4. **외감 여부가 중요한 신뢰도 지표**
   - 외감 기업 vs 비외감 기업 부도율 차이
   - 한국 시장 특수성 반영

### ⚠️ 한계 (Part 1의 한계점)

1. **개별 변수로는 예측력 제한적**
   - 단변량 AUC < 0.7 (참고: 실무에서는 0.8+ 필요)
   - 유동비율만으로는 부도를 정확히 예측 불가

2. **변수 간 상호작용 미고려**
   - 유동성 + 수익성의 조합은?
   - 부채비율이 높아도 이자보상배율이 높으면?

3. **비선형 관계 미탐색**
   - 부채비율 50% vs 200%의 리스크 차이는 선형이 아님
   - 임계점(threshold) 존재 가능

### ➡️ 다음 단계: **Part 2 - 도메인 지식 기반 복합 특성 생성**

#### 1. 유동성 위기 지표 (10개 특성)
- **즉각지급능력** = 현금 / 단기차입금
- **현금소진일수** = 현금 / (영업비용 / 365)
- **운전자본 건전성** = (유동자산 - 유동부채) / 총자산
- **긴급유동성** = (현금 + 단기금융자산) / 유동부채

#### 2. 지급불능 패턴 (8개 특성)
- **자본잠식도** = 자본 / 총자산 (음수면 완전 잠식)
- **이자보상배율** = 영업이익 / 이자비용
- **부채상환년수** = 부채 / 영업현금흐름
- **재무레버리지** = 총자산 / 자본

#### 3. 재무조작 탐지 (15개 특성)
- **한국형 M-Score** (Beneish M-Score 변형)
- **매출채권 이상지표** = (매출채권 / 매출) 증가율
- **재고 이상지표** = (재고 / 매출원가) 증가율
- **발생액 품질** = (당기순이익 - 영업현금흐름) / 총자산

#### 4. 한국 시장 특화 (13개 특성)
- **대기업 의존도** = 상위 3개 거래처 매출 / 총매출
- **제조업 리스크** = 업종 더미 × 재고/유동자산 비율
- **외감 여부** (0/1)
- **업력** = 2021 - 설립연도

#### 5. 이해관계자 행동 (9개 특성)
- **연체 정보** (기업신용공여연체 등)
- **세금체납** (국세, 지방세)
- **신용등급** (구간화)
- **이해관계자 불신지수** = 연체 + 체납 + 신용등급

#### 6. 복합 리스크 지표 (7개 특성)
- **종합부도위험스코어** = 가중합 (유동성 + 수익성 + 부채)
- **조기경보신호수** = 임계값 초과 지표 개수
- **재무건전성지수** = PCA 또는 요인분석

#### 7. 상호작용/비선형 (3개 특성)
- **레버리지 × 수익성** = 부채비율 × ROA
- **부채비율 제곱** (비선형 리스크)
- **임계값 기반 특성** (예: 이자보상배율 < 1)

### 🎯 최종 목표

- **65개 도메인 특성** 생성 (Part 2)
- **앙상블 모델** 학습 (LightGBM, XGBoost, CatBoost) (Part 4)
- **PR-AUC 0.8+**, **F2-Score 0.7+**, **Type II Error < 20%** 달성 (Part 5)

---

## 🚀 Part 2에서 계속...

다음 노트북: **발표_Part2_도메인_특성_공학.ipynb**

